In [5]:
import coiled
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
cluster = coiled.Cluster(
    name="github-json",
    software="matthew-powers/my_env", 
    n_workers=10,
)

Output()

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-matthew24-148591-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-148591-firewall -> coiled-dask-matthew24-148591-firewall]
Created FW rules: coiled-dask-matthew24-148591-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-148591-cluster-firewall -> coiled-dask-matthew24-148591-cluster-firewall]
Created scheduler VM: coiled-dask-matthew24-148591-scheduler (type: t3.medium, ip: ['44.203.228.185'])


In [3]:
client = Client(cluster)

In [6]:
# read data in without dtypes
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2010-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    on_bad_lines="skip",
    dtype={
        'tolls_amount': 'float64',
        "store_and_fwd_flag": "object",
    },
)

In [7]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [8]:
len(ddf)

168994353

In [9]:
ddf.describe().compute()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
count,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689942e+08,1.689942e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08
mean,1.674221e+00,5.864681e+00,-7.239051e+01,3.988347e+01,1.032463e+00,-7.241661e+01,3.989821e+01,9.844589e+00,3.221727e-01,4.955940e-01,7.605548e-01,-7.496016e-02,1.135107e+01
std,1.300666e+00,5.409394e+03,1.102328e+01,7.053625e+00,4.236116e-01,1.089876e+01,7.027043e+00,1.664848e+03,3.693486e-01,1.345498e-01,1.735413e+02,2.336194e+03,2.873962e+03
min,0.000000e+00,-2.147483e+07,-3.509015e+03,-3.579139e+03,0.000000e+00,-3.579139e+03,-3.538432e+03,-2.147481e+07,-1.000000e+00,-1.000000e+00,-1.677720e+06,-2.147484e+07,-2.147483e+07
25%,1.000000e+00,1.090000e+00,-7.399139e+01,4.073673e+01,1.000000e+00,-7.399071e+01,4.073553e+01,5.700000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,7.000000e+00
50%,1.000000e+00,1.800000e+00,-7.398101e+01,4.075392e+01,1.000000e+00,-7.397925e+01,4.075437e+01,8.100000e+00,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,9.540000e+00
75%,3.000000e+00,3.250000e+00,-7.396582e+01,4.076844e+01,1.000000e+00,-7.396257e+01,4.076894e+01,1.210000e+01,5.000000e-01,5.000000e-01,1.600000e+00,0.000000e+00,1.380000e+01
max,2.550000e+02,1.620163e+07,3.569931e+03,3.377993e+03,2.210000e+02,3.443651e+03,3.510381e+03,9.396007e+04,6.157800e+02,1.311220e+03,9.380200e+02,5.510070e+03,9.396057e+04


In [10]:
# repartition and write 2010 data to Parquet 
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/tmp/matt/nyc-tlc-2010",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]